In this notebook, the practice of the lifetime package and estimate customer lifetime value is conducted.

Reference: 
1. clv learning: https://www.youtube.com/watch?v=486x8ccQThE
2. dataset used: https://www.kaggle.com/datasets/nathaniel/uci-online-retail-ii-data-set also available at [UCI Repo] (https://archive.ics.uci.edu/dataset/502/online+retail+ii)
3. notebook looked as helping hand: https://www.kaggle.com/code/halimedogan/crm-analytics-cltv/notebook

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import summary_data_from_transaction_data as summary
from lifetimes.utils import _customer_lifetime_value 

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")
sns.set_palette("pastel")

In [2]:
import os
print(os.getcwd())

c:\Users\Monika\Projects\Customer Life-time Value\Learning\clv_practice1_uci


## Load data

In [3]:
df1 = pd.read_excel(r"uci_online_retail_II.xlsx", sheet_name="Year 2009-2010")
df2 = pd.read_excel(r"uci_online_retail_II.xlsx", sheet_name="Year 2010-2011")

In [4]:
df1.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [5]:
df2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
def check_df(dataframe):
    print("################ Shape ####################")
    print(dataframe.shape)
    print("############### Types #####################")
    print(dataframe.dtypes)
    print("############### Unique Value Counts ###################")
    print(dataframe.nunique())
    print("############### NA Counts ########################")
    print(dataframe.isnull().sum())

In [7]:
check_df(df1)

################ Shape ####################
(525461, 8)
############### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
############### Unique Value Counts ###################
Invoice        28816
StockCode       4632
Description     4681
Quantity         825
InvoiceDate    25296
Price           1606
Customer ID     4383
Country           40
dtype: int64
############### NA Counts ########################
Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64


In [8]:
check_df(df2)

################ Shape ####################
(541910, 8)
############### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
############### Unique Value Counts ###################
Invoice        25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
Price           1630
Customer ID     4372
Country           38
dtype: int64
############### NA Counts ########################
Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64


## analysis of data values
- to create dataframe compatible with lifetimes library

In [35]:
df1_nullCID_Invoice = df1.loc[df1['Customer ID'].isna()]['Invoice'].unique()
df2_nullCID_Invoice = df2.loc[df2['Customer ID'].isna()]['Invoice'].unique()

print("Number of unique invoices with null Customer ID in dataframe:")
print(f"df1: {df1_nullCID_Invoice.shape[0]} out of {df1['Invoice'].nunique()}")
print(f"df2: {df2_nullCID_Invoice.shape[0]} out of {df2['Invoice'].nunique()}")


Number of unique invoices with null Customer ID in dataframe:
df1: 5229 out of 28816
df2: 3710 out of 25900


In [ ]:
df1_withCID_Invoice_record = df1.loc[~df1['Customer ID'].isna()]['Invoice']

df2_withCID_Invoice_record = df2.loc[~df2['Customer ID'].isna()]['Invoice']

print('Count of invoices with a Customer ID also appear without a Customer ID')
print('df1:', 
        df1_withCID_Invoice_record.isin(df1_nullCID_Invoice).sum())

print('df2:',
        df2_withCID_Invoice_record.isin(df2_nullCID_Invoice).sum())

Count of invoices with a Customer ID also appear without a Customer ID
df1: 0
df2: 0


Given the above, we can conclude that the null Customer IDs cannot be imputed and cannot be used for analysis.

In [ ]:
df1.groupby('Invoice')[['Customer ID']].nunique().eq(1).all(axis=1).sum()

np.int64(23587)

In [42]:
df1.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### unique invoice record

In [218]:
def transform_data(data):
    data = data.copy()
    data = data[~data['Customer ID'].isna()]
    data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
    data['Invoice'] = data['Invoice'].astype(str)
    data['Customer ID'] = data['Customer ID'].astype(str)
    data['Amount'] = data['Quantity'] * data['Price']
    
    # aggregate template
    agg_dict = {
        'InvoiceDate': lambda x: x.iloc[0] if x.nunique() == 1 else x.max(),
        'Customer ID': lambda x: x.iloc[0] if x.nunique() == 1 else 'Multiple',
        'Country': lambda x: x.iloc[0] if x.nunique() == 1 else 'Multiple',
        'Amount': 'sum',
        'Quantity': list,
        'Price': list,
        'StockCode': list,
        'Description': list
    }
    grouped = data.groupby('Invoice', as_index=False).agg(agg_dict)

    # Add new columns after aggregation
    grouped['TotalItems'] = grouped['Quantity'].apply(sum)
    grouped['NumUniqueProd'] = grouped['StockCode'].apply(len)
    grouped['AvgPrice'] = grouped['Price'].apply(lambda x: round(sum(x)/len(x), 2))
    
    return grouped

In [219]:
df1_summary = transform_data(df1)
df2_summary = transform_data(df2)

In [220]:
print('multiple customer IDs in invoice:', df1_summary[df1_summary['Customer ID'] == 'Multiple'].shape[0])
print('multiple country in invoice:', df2_summary[df2_summary['Country'] == 'Multiple'].shape[0])

multiple customer IDs in invoice: 0
multiple country in invoice: 0


In [214]:
df1_summary.tail()

,Invoice,InvoiceDate,Customer ID,Country,Amount,Quantity,Price,StockCode,Description,TotalItems,NumUniqueProd,AvgPrice
23582,C538121,2010-12-09 15:36:00,15535.0,United Kingdom,-12.75,[-1],[12.75],[22461],[SAVOY ART DECO CLOCK],-1,1,12.75
23583,C538122,2010-12-09 15:38:00,14696.0,United Kingdom,-1.25,[-1],[1.25],[22444],[GROW YOUR OWN PLANT IN A CAN ],-1,1,1.25
23584,C538123,2010-12-09 15:41:00,12605.0,Germany,-7.50,"[-1, -1, -2]","[1.65, 1.65, 2.1]","[22331, 22333, 22956]","[WOODLAND PARTY BAG + STICKER SET, RETROSPOT P...",-4,3,1.80
23585,C538124,2010-12-09 15:43:00,15329.0,United Kingdom,-17.70,"[-4, -1, -1]","[0.5, 2.95, 12.75]","[M, 22699, 22423]","[Manual, ROSES REGENCY TEACUP AND SAUCER , REG...",-6,3,5.40
23586,C538164,2010-12-09 17:32:00,14031.0,United Kingdom,-1.95,[-1],[1.95],[35004B],[SET OF 3 BLACK FLYING DUCKS],-1,1,1.95


In [73]:
df1_summary[df1_summary['TotalItems'] < 0]['Invoice'].astype(str).str.startswith('C').sum()

np.int64(4372)

In [74]:
df1_summary['Invoice'].astype(str).str[0].unique()

array(['4', '5', 'C'], dtype=object)

In [75]:
df1_summary[df1_summary['TotalItems'] > 0]['Invoice'].astype(str).str.startswith('C').sum()

np.int64(0)

In [ ]:
df1_summary[df1_summary['TotalItems'] > 0]['Invoice'].str[0].str.isdigit().sum()

np.int64(19215)

In [107]:
df1_summary[df1_summary['TotalItems'] < 0]['Invoice'].str[0].str.isdigit().sum()

np.int64(0)

The invoice record consist of negative order value and have C prefix in Invoice ID.

Such invoices may indicate one or more of the following:
1. loss incur by the business in handling and processing the order, part of CAC
2. these are return orders for some old original orders
3. these are product purchase by the business (online retailer)

In [104]:
df1_summary[df1_summary['TotalItems'] < 0]['Invoice'].str[1:]


19215    489449
19216    489459
19217    489476
19218    489503
19219    489504
          ...  
23582    538121
23583    538122
23584    538123
23585    538124
23586    538164
Name: Invoice, Length: 4372, dtype: object

In [97]:
df1_summary['Invoice'].isin([df1_summary[df1_summary['TotalItems'] < 0]['Invoice'].astype(str).str[1:]]).sum()

np.int64(0)

In [127]:
df2_summary[df2_summary['TotalItems'] < 0]['Invoice'].str[1:]

18536    536379
18537    536383
18538    536391
18539    536506
18540    536543
          ...  
22185    581484
22186    581490
22187    581499
22188    581568
22189    581569
Name: Invoice, Length: 3654, dtype: object

In [ ]:
df1_summary['Invoice'].isin([df2_summary[df2_summary['TotalItems'] < 0]['Invoice'].str[1:]]).sum()

np.int64(0)

In [130]:
df2_summary['Invoice'].isin(df1_summary['Invoice']).sum()

np.int64(901)

df1 and df2 has some common invoice number.

In [133]:
df1_summary['Invoice'].isin(df2_summary['Invoice'])

0        False
1        False
2        False
3        False
4        False
         ...  
23582     True
23583     True
23584     True
23585     True
23586     True
Name: Invoice, Length: 23587, dtype: bool

### 1st attempt
- replace multiple invoice date with oldest one.

In [152]:
common_invoices = set(df1_summary['Invoice']) & set(df2_summary['Invoice'])
df1_common = df1_summary[df1_summary['Invoice'].isin(common_invoices)]
df2_common = df2_summary[df2_summary['Invoice'].isin(common_invoices)]

len(common_invoices)

901

In [168]:
df1_common.head()

,Invoice,InvoiceDate,Customer ID,Country,Amount,Quantity,Price,StockCode,Description,TotalItems,NumUniqueProd,AvgPrice
18439,536365,2010-12-01 08:26:00,17850.0,United Kingdom,139.12,"[6, 6, 8, 6, 6, 2, 6]","[2.55, 3.39, 2.75, 3.39, 3.39, 7.65, 4.25]","[85123A, 71053, 84406B, 84029G, 84029E, 22752,...","[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...",40,7,3.91
18440,536366,2010-12-01 08:28:00,17850.0,United Kingdom,22.20,"[6, 6]","[1.85, 1.85]","[22633, 22632]","[HAND WARMER UNION JACK, HAND WARMER RED POLKA...",12,2,1.85
18441,536367,2010-12-01 08:34:00,13047.0,United Kingdom,278.73,"[32, 6, 6, 8, 6, 6, 3, 2, 3, 3, 4, 4]","[1.69, 2.1, 2.1, 3.75, 1.65, 4.25, 4.95, 9.95,...","[84879, 22745, 22748, 22749, 22310, 84969, 226...","[ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO...",83,12,4.85
18442,536368,2010-12-01 08:34:00,13047.0,United Kingdom,70.05,"[6, 3, 3, 3]","[4.25, 4.95, 4.95, 4.95]","[22960, 22913, 22912, 22914]","[JAM MAKING SET WITH JARS, RED COAT RACK PARIS...",15,4,4.78
18443,536369,2010-12-01 08:35:00,13047.0,United Kingdom,17.85,[3],[5.95],[21756],[BATH BUILDING BLOCK WORD],3,1,5.95


In [169]:
df2_common.head()

,Invoice,InvoiceDate,Customer ID,Country,Amount,Quantity,Price,StockCode,Description,TotalItems,NumUniqueProd,AvgPrice
0,536365,2010-12-01 08:26:00,17850.0,United Kingdom,139.12,"[6, 6, 8, 6, 6, 2, 6]","[2.55, 3.39, 2.75, 3.39, 3.39, 7.65, 4.25]","[85123A, 71053, 84406B, 84029G, 84029E, 22752,...","[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...",40,7,3.91
1,536366,2010-12-01 08:28:00,17850.0,United Kingdom,22.20,"[6, 6]","[1.85, 1.85]","[22633, 22632]","[HAND WARMER UNION JACK, HAND WARMER RED POLKA...",12,2,1.85
2,536367,2010-12-01 08:34:00,13047.0,United Kingdom,278.73,"[32, 6, 6, 8, 6, 6, 3, 2, 3, 3, 4, 4]","[1.69, 2.1, 2.1, 3.75, 1.65, 4.25, 4.95, 9.95,...","[84879, 22745, 22748, 22749, 22310, 84969, 226...","[ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO...",83,12,4.85
3,536368,2010-12-01 08:34:00,13047.0,United Kingdom,70.05,"[6, 3, 3, 3]","[4.25, 4.95, 4.95, 4.95]","[22960, 22913, 22912, 22914]","[JAM MAKING SET WITH JARS, RED COAT RACK PARIS...",15,4,4.78
4,536369,2010-12-01 08:35:00,13047.0,United Kingdom,17.85,[3],[5.95],[21756],[BATH BUILDING BLOCK WORD],3,1,5.95


In [177]:
print(f"df1 invoice date, min: {df1_summary['InvoiceDate'].min()}, max: {df1_summary['InvoiceDate'].max()}")
print(f"df2 invoice date, min: {df2_summary['InvoiceDate'].min()}, max: {df2_summary['InvoiceDate'].max()}")
print(f"common invoice date, min: {df1_common['InvoiceDate'].min()}, max: {df1_common['InvoiceDate'].max()}")

df1 invoice date, min: 2009-12-01 07:45:00, max: 2010-12-09 20:01:00
df2 invoice date, min: 2010-12-01 08:26:00, max: 2011-12-09 12:50:00
common invoice date, min: 2010-12-01 08:26:00, max: 2010-12-09 20:01:00


In [183]:
start = pd.to_datetime('2010-12-01 08:26:00')
end = pd.to_datetime('2010-12-09 20:01:00')
print(df1_summary[(df1_summary['InvoiceDate'] >= start) & (df1_summary['InvoiceDate'] <= end)].shape[0])
print(df2_summary[(df2_summary['InvoiceDate'] >= start) & (df2_summary['InvoiceDate'] <= end)].shape[0])

900
900


In [186]:
df2_summary[df2_summary['InvoiceDate'] > end]['InvoiceDate'].min()

Timestamp('2010-12-10 09:33:00')

df1 and df2 have 900 invoice records with overlapping dates, and but 901 in overlapping invoice.

In [194]:
df1_dateoverlap = df1_summary[(df1_summary['InvoiceDate'] >= start) & (df1_summary['InvoiceDate'] <= end)]
df2_dateoverlap = df2_summary[(df2_summary['InvoiceDate'] >= start) & (df2_summary['InvoiceDate'] <= end)]

df1_common[~df1_common['Invoice'].isin(df1_dateoverlap['Invoice'])]

,Invoice,InvoiceDate,Customer ID,Country,Amount,Quantity,Price,StockCode,Description,TotalItems,NumUniqueProd,AvgPrice
18555,536591,NaT,14606.0,United Kingdom,198.32,"[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2.1, 0.42, 1.25, 1.25, 1.25, 1.25, 1.95, 2.1,...","[21427, 22135, 90214M, 90214V, 90214S, 90214J,...","[SKULLS STORAGE BOX SMALL, LADLE LOVE HEART PI...",93,40,2.95


In [215]:
df1_summary[df1_summary['InvoiceDate'].isna()].head()

,Invoice,InvoiceDate,Customer ID,Country,Amount,Quantity,Price,StockCode,Description,TotalItems,NumUniqueProd,AvgPrice
1429,492807,NaT,17211.0,United Kingdom,342.82,"[1, 2, 4, 3, 3, 12, 1, 6, 4, 6, 12, 2, 2, 2, 2...","[1.25, 1.25, 1.25, 1.25, 1.25, 1.65, 3.75, 4.9...","[85131A, 85129A, 85129D, 85129B, 85129A, 21645...","[BEADED PEARL HEART WHITE ON STICK, WHITE PEAR...",177,30,2.68
3778,499967,NaT,16636.0,United Kingdom,1311.45,"[10, 10, 20, 20, 20, 15, 20, 20, 10, 20, 20, 4...","[6.35, 6.35, 0.85, 0.85, 1.65, 2.95, 1.95, 1.9...","[21524, 48185, 20723, 20724, 20725, 21937, 850...","[DOOR MAT SPOTTY HOME SWEET HOME, DOOR MAT FAI...",795,31,2.47
3970,500353,NaT,12668.0,Germany,1966.84,"[12, 10, 4, 6, 2, 3, 24, 24, 24, 18, 6, 6, 6, ...","[1.65, 1.25, 3.75, 2.1, 9.95, 9.95, 0.55, 0.55...","[22136, 21154, 22073, 20754, 79072, 21217, 212...","[LOVE HEART SOCK HANGER, RED SPOTTY OVEN GLOVE...",1114,110,2.79
4418,501618,NaT,15819.0,United Kingdom,296.42,"[10, 10, 10, 36, 6, 16, 12, 12, 10, 3, 3, 8, 1...","[1.95, 0.85, 1.65, 0.85, 2.55, 1.25, 0.85, 0.8...","[20712, 20719, 20726, 22610, 22353, 20977, 215...","[JUMBO BAG WOODLAND ANIMALS, WOODLAND CHARLOTT...",198,19,1.97
4554,501871,NaT,14970.0,United Kingdom,658.00,"[12, 4, 10, 10, 10, 10, 2, 6, 6, 6, 6, 6, 24, ...","[1.65, 7.95, 6.35, 6.35, 6.35, 6.35, 2.1, 0.65...","[85206A, 21777, 21523, 22366, 48187, 48138, 84...","[CREAM FELT EASTER EGG BASKET, RECIPE BOX WITH...",146,15,3.88


In [ ]:
df1.query('Invoice == 492807')

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
41630,492807,85131A,BEADED PEARL HEART WHITE ON STICK,1,2009-12-20 12:28:00,1.25,17211.0,United Kingdom
41631,492807,85129A,"WHITE PEARL BEADED HEART, SMALL",2,2009-12-20 12:28:00,1.25,17211.0,United Kingdom
41632,492807,85129D,BEADED CRYSTAL HEART PINK SMALL,4,2009-12-20 12:28:00,1.25,17211.0,United Kingdom
41633,492807,85129B,BEADED CRYSTAL HEART GREEN SMALL,3,2009-12-20 12:28:00,1.25,17211.0,United Kingdom
41634,492807,85129A,"WHITE PEARL BEADED HEART, SMALL",3,2009-12-20 12:28:00,1.25,17211.0,United Kingdom
41635,492807,21645,ASSORTED TUTTI FRUTTI ROUND BOX,12,2009-12-20 12:28:00,1.65,17211.0,United Kingdom
41636,492807,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,1,2009-12-20 12:28:00,3.75,17211.0,United Kingdom
41637,492807,21813,GARLAND WITH STARS AND BELLS,6,2009-12-20 12:28:00,4.95,17211.0,United Kingdom
41638,492807,22139,RETRO SPOT TEA SET CERAMIC 11 PC,4,2009-12-20 12:28:00,4.95,17211.0,United Kingdom
41639,492807,37340,MULTICOLOUR SPRING FLOWER MUG,6,2009-12-20 12:28:00,0.85,17211.0,United Kingdom


#### jump to unique invoice record

### 2nd attempt

In [221]:
common_invoices = set(df1_summary['Invoice']) & set(df2_summary['Invoice'])
df1_common = df1_summary[df1_summary['Invoice'].isin(common_invoices)]
df2_common = df2_summary[df2_summary['Invoice'].isin(common_invoices)]

len(common_invoices)

901

In [222]:
start = pd.to_datetime('2010-12-01 08:26:00')
end = pd.to_datetime('2010-12-09 20:01:00')
print(df1_summary[(df1_summary['InvoiceDate'] >= start) & (df1_summary['InvoiceDate'] <= end)].shape[0])
print(df2_summary[(df2_summary['InvoiceDate'] >= start) & (df2_summary['InvoiceDate'] <= end)].shape[0])

901
901


Now, all the 901 invoice dates are overlapping whose invoice IDs are common in both dataframes.
We can drop the 901 invoices from df1 or df2.

In [233]:
df_combine = pd.concat([df1_summary, df2_summary])
df_combine = df_combine.drop_duplicates(subset='Invoice', keep='first')  # or 'last'
df_combine.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44876 entries, 0 to 22189
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Invoice        44876 non-null  object        
 1   InvoiceDate    44876 non-null  datetime64[ns]
 2   Customer ID    44876 non-null  object        
 3   Country        44876 non-null  object        
 4   Amount         44876 non-null  float64       
 5   Quantity       44876 non-null  object        
 6   Price          44876 non-null  object        
 7   StockCode      44876 non-null  object        
 8   Description    44876 non-null  object        
 9   TotalItems     44876 non-null  int64         
 10  NumUniqueProd  44876 non-null  int64         
 11  AvgPrice       44876 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(7)
memory usage: 4.5+ MB
